# Ανάλυση Κατασκεύων σε Η/Υ - Άσκηση 1 (Μαρτ.2021)

In [8]:
from streng.ppp.sections.geometry.rectangular import RectangularSectionGeometry
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import openseespy.opensees as ops
import openseespy.postprocessing.ops_vis as opsv

## Εισαγωγή αριθμού μητρώου φοιτητών

Αντικαταστήστε τα ψηφία με αυτά του αριθμού μητρώου σας και τρέξτε το jupyter notebook.  
Δε θα χρειαστεί να κάνετε κάποια άλλη αλλαγή ή κάποια εισαγωγή δεδομένων

In [9]:
κ, λ, μ, ν = 9, 8, 7, 6

## Δεδομένα

In [10]:
# Δεδομένα φοιτητών βάσει ΑΕΜ
h=3.0+0.1*(κ+μ)
a=b=0.40
F=110.0+μ+ν
M=70.0+λ+ν
E=(2+0.1*ν)*10**7
U = 0.2

# # Δεδομένα παραδείγματος βιβλίου
# h=4.0
# a=b=0.30
# F=100.0
# M=80.0
# E=2.8*10**7
# U = 0.2

In [11]:
print(f'h = {h:.2f}m')
print(f'a = b = {a:.2f}m')
print(f'F = {F:.2f}kN')
print(f'M = {M:.2f}kNm')
print(f'E = {E:.0f}kPa')

h = 4.60m
a = b = 0.40m
F = 123.00kN
M = 84.00kNm
E = 26000000kPa


## Επίλυση με το OpenSees

In [12]:
ops.wipe()
ops.model('basic', '-ndm', 2, '-ndf', 3)

U = 0.2
G = E / (2.0*(1.0+U))

rect_col = RectangularSectionGeometry(b=b, h=a)
A_col = rect_col.area
Iz_col = rect_col.moment_of_inertia_xx # * 0.5
Avy_col = rect_col.shear_area_2 # * 0.5

ops.node(1, 0., 0.)
ops.node(2, 0., h)

ops.fix(1, 1, 1, 1)

ops.geomTransf('Linear', 1)

ops.element('ElasticTimoshenkoBeam', 1, 1, 2, E, G, A_col, Iz_col, Avy_col, 1)  # , '-mass', 0., '-lMass')

results_ele = []
results_node = []

# F Analysis
ops.timeSeries('Constant', 1)
ops.pattern('Plain', 1, 1)
ops.load(2, F, 0., 0.)
ops.constraints('Transformation')
ops.numberer('RCM')
ops.system('BandGeneral')
ops.test('NormDispIncr', 1.0e-6, 6, 2)
ops.algorithm('Linear')
ops.integrator('LoadControl', 1)
ops.analysis('Static')
ops.analyze(1)

results_ele.append(['F', ops.eleForce(1)])
results_node.append(['F', ops.nodeDisp (2)])

ops.remove('timeSeries', 1)
ops.remove('loadPattern', 1)
ops.wipeAnalysis()

# M Analysis
ops.timeSeries('Constant', 1)
ops.pattern('Plain', 1, 1)
ops.load(2, 0., 0., -M)
ops.constraints('Transformation')
ops.numberer('RCM')
ops.system('BandGeneral')
ops.test('NormDispIncr', 1.0e-6, 6, 2)
ops.algorithm('Linear')
ops.integrator('LoadControl', 1)
ops.analysis('Static')
ops.analyze(1)

results_ele.append(['M', ops.eleForce(1)])
results_node.append(['M', ops.nodeDisp (2)])

ops.remove('timeSeries', 1)
ops.remove('loadPattern', 1)
ops.wipeAnalysis()

results_ele.append(['F+M', np.add(results_ele[0][1], results_ele[1][1])])
results_node.append(['F+M', np.add(results_node[0][1], results_node[1][1])])

print(results_ele)


[['F', [-122.99999999999994, 0.0, 565.7999999999997, 122.99999999999994, 0.0, 0.0]], ['M', [-4.263256414560601e-14, 0.0, 84.0000000000002, 4.263256414560601e-14, 0.0, -84.00000000000003]], ['F+M', array([-123. ,    0. ,  649.8,  123. ,    0. ,  -84. ])]]


## Εμφάνιση αποτελεσμάτων

### Εντατικά μεγέθη

In [13]:
# Τα αποτελέσματα έχουν τη μορφή [-V2(i), N(i), M3(i), V2(j), N(j), M3(j)]
# for re in results_ele:
#     print(f'{re[0]}: {np.round(re[1],2)}')
    
for re in results_ele:
    print(f'Για φόρτιση {re[0]}: M={re[1][2]:.2f}kNm')
    print(f'Για φόρτιση {re[0]}: V={re[1][3]:.2f}kN')
    print()

Για φόρτιση F: M=565.80kNm
Για φόρτιση F: V=123.00kN

Για φόρτιση M: M=84.00kNm
Για φόρτιση M: V=0.00kN

Για φόρτιση F+M: M=649.80kNm
Για φόρτιση F+M: V=123.00kN



### Μετακινήσεις

In [14]:
# Τα αποτελέσματα έχουν τη μορφή [Trans1, Trans3, -Rotn2]
# for re in results_node:
#     print(f'{re[0]}: {np.round(re[1],4)}')

for re in results_node:
    print(f'Για φόρτιση {re[0]}: δ={100*re[1][0]:.2f}cm')
    print()

Για φόρτιση F: δ=7.23cm

Για φόρτιση M: δ=1.60cm

Για φόρτιση F+M: δ=8.84cm

